In [3]:

import numpy as np
import random
#%%
%load_ext autoreload
%autoreload 2

In [4]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        print(self.sizes[1:])
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    # TODO: understand & run examples
    def SGD(self, training_data: list, epochs: int, mini_batch_size: int, eta: float,
            test_data):
        """
        Train the neural network using mini-batch stochastic gradient descent.

        Parameters:
            training_data (list of tuples) - (x,y)[]: A list where each tuple (x, y) corresponds to training inputs and their desired outputs.
            epochs (int): The number of epochs to train the network.
            mini_batch_size (int): The size of each mini batch for the gradient descent to use when sampling?
            eta (float): The learning rate. η
            test_data (list of tuples, optional): Data against which the network is evaluated after each epoch.If provided, it allows for tracking progress but may slow down the training.

        """
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    # TODO: understand & run examples
    def update_mini_batch(self, mini_batch: list, eta: float):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        
        Parameters:
            mini_batch (list of tuples): Each tuple (x, y) represents training inputs and their desired outputs.
            eta (float): The learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [
            w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)
        ]
        self.biases = [
            b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)
        ]

    # TODO: black box for now
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives partial C_x /
        partial a for the output activations."""
        return output_activations - y


def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))

In [5]:
# net = Network([2, 3, 1])
# print(net.biases)
# print(net.weights)
# print(net.weights[1])





In [26]:

# loading data
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
print("training_data", training_data)
print("validation_data", validation_data)
print("test_data", test_data)



training_data <zip object at 0x13695c3c0>
validation_data <zip object at 0x13695ddc0>
test_data <zip object at 0x13695c680>


In [27]:
# Eval
net = Network([784, 10])

net.SGD(training_data, epochs=30, mini_batch_size=10, eta=3.0, test_data=test_data)

[10]
Epoch 0: 6382 / 10000
Epoch 1: 7367 / 10000
Epoch 2: 7426 / 10000
Epoch 3: 8295 / 10000
Epoch 4: 8354 / 10000
Epoch 5: 8337 / 10000
Epoch 6: 8345 / 10000
Epoch 7: 8357 / 10000
Epoch 8: 8334 / 10000
Epoch 9: 8365 / 10000
Epoch 10: 8361 / 10000
Epoch 11: 8391 / 10000
Epoch 12: 8391 / 10000
Epoch 13: 8377 / 10000
Epoch 14: 8382 / 10000
Epoch 15: 8347 / 10000
Epoch 16: 8361 / 10000
Epoch 17: 8363 / 10000
Epoch 18: 8394 / 10000
Epoch 19: 8348 / 10000
Epoch 20: 8350 / 10000
Epoch 21: 8371 / 10000
Epoch 22: 8384 / 10000
Epoch 23: 8382 / 10000
Epoch 24: 8377 / 10000
Epoch 25: 8386 / 10000
Epoch 26: 8400 / 10000
Epoch 27: 8382 / 10000
Epoch 28: 8404 / 10000
Epoch 29: 8399 / 10000
